In [1]:
import cv2
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd 
import time
# %matplotlib inline
%matplotlib qt

import plotly.express as px

In [2]:
imagePath = "/home/swarup/Desktop/nasa/images/SSMI_sea_ice_minimum_no_dates."
yearRange = range(1979,2021)
imageFormat = ".tif"
images = [imagePath + str(year) + imageFormat for year in yearRange]

In [8]:
s = cv2.imread(imagePath+ str(1979) + imageFormat)
plt.imshow(s);
# ice = cv2.inRange(s,lowerb=lower,upperb=upper)
# plt.imshow(ice);

In [3]:
lower, upper = np.array([153,161,165]), np.array([255,255,255])

In [4]:
def calcIcePerc(image):
    
    img = cv2.imread(image)
    imgSize = img.size
    spaceSize = img[img==0].size 
    earthSize = imgSize - spaceSize

    ice = cv2.inRange(img,lowerb=lower,upperb=upper)
    # mask = ice.ravel()==255
    iceSize = ice.ravel()[ice.ravel()==255].size
    icePercent = iceSize/earthSize
    icePercent *= 100

    return icePercent

In [5]:
tic = time.perf_counter()
icePercentages = [calcIcePerc(image) for image in images]
toc = time.perf_counter()

print(f"Time to calc all the percentages: {toc-tic:.4f}")

Time to calc all the percentages: 6.9405


In [6]:
def calcIcePercGS(image):
    img = cv2.imread(image,0)
    imgSize = img.size 
    spaceSize = img[img==0].size 
    earthSize= imgSize - spaceSize

    ice = cv2.inRange(img,150,255)
    iceSize = ice.ravel()[ice.ravel()==255].size
    icePercent = iceSize/earthSize
    icePercent *= 100

    return icePercent

In [7]:
tic = time.perf_counter()
icePercentagesGS = [calcIcePercGS(image) for image in images]
toc = time.perf_counter()

print(f"Time to calc all the percentages: {toc-tic:.4f}")

Time to calc all the percentages: 4.1272


In [8]:
icePercentages_DF = pd.DataFrame(data={"year":yearRange,"icePercentages":icePercentages,})
icePercentagesGS_DF = pd.DataFrame(data={"year":yearRange,"icePercentagesGS":icePercentagesGS})

px.line(data_frame=icePercentages_DF,x="year",y="icePercentages")

In [9]:
global_temp = pd.read_csv('annualglobaltemp.txt',skiprows=4)
global_temp.tail()

,----------------------------
136,2016 1.02 0.88
137,2017 0.93 0.91
138,2018 0.85 0.95
139,2019 0.98 0.98
140,2020 1.02 1.01


In [14]:
temperatures = [float(row[0][9:14]) for row in global_temp.values]
temperatures = temperatures[-42:]
temperatures[-5:]

[1.02, 0.93, 0.85, 0.98, 1.02]

In [18]:
DF = pd.DataFrame({'Year':range(1979,2021),"Ice %": icePercentages,"temp":temperatures})
print(DF.head())
print(DF.tail())

   Year     Ice %  temp
0  1979  8.626372  0.16
1  1980  8.791269  0.26
2  1981  8.341235  0.32
3  1982  8.634605  0.14
4  1983  8.829432  0.31
    Year     Ice %  temp
37  2016  5.926026  1.02
38  2017  6.832128  0.93
39  2018  7.003648  0.85
40  2019  6.511545  0.98
41  2020  6.262098  1.02


In [20]:
px.line(data_frame=DF,x='Year',y='Ice %')

In [19]:
px.line(data_frame=DF,x="Year",y="temp")

In [26]:
DF.to_excel('79-20_temp.xlsx')
# DF